# Goal:

provide a plaintext interface to analyze step-by-step what is happening in the encrypted code

In [19]:
import numpy as np
np.random.seed(42)
import pandas as pd

np.seterr(all='raise')

DEBUG = True

# Load and Process the Data

In [20]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def load_data(num_samples, compare_to_r_ref):
    x_file = "../data/X_norm_1024.csv"
    y_file = "../data/y_1024.csv"
    train_x = pd.read_csv(x_file)
    train_x = train_x.to_numpy()[:num_samples]
    train_y = pd.read_csv(y_file)
    train_y = train_y.to_numpy()[:num_samples]
    print(f"{bcolors.OKGREEN}Using subsampled data to compare Python-C++{bcolors.ENDC}")
    print(f"{bcolors.OKGREEN}Reading in {x_file}, {y_file} {bcolors.ENDC}")

    print(f"Train X shape is: {train_x.shape}")
    print(f"Train y shape is: {train_y.shape}")
    return train_x, train_y

In [21]:
NUM_SAMPLES = 1024
COMPARE_TO_R_REF = False
lr = 0.1
mu = 0.1
train_x, train_y = load_data(
    num_samples=NUM_SAMPLES,
    compare_to_r_ref=COMPARE_TO_R_REF
)
betas = np.zeros((10, ))

Using subsampled data to compare Python-C++
Reading in ../data/X_norm_1024.csv, ../data/y_1024.csv 
Train X shape is: (1024, 10)
Train y shape is: (1024, 1)


In [24]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def fwd(train_x, betas, dbg=False):
    preds = train_x @ betas
    return np.expand_dims(sigmoid(preds), -1)

def calculate_gradient(train_x, train_y, betas, fwd, dbg):
    preds = fwd(train_x, betas, dbg)
    gradient = -train_x.T @ (train_y - preds) / len(train_y)
    return gradient

def cost(x, y, theta):
    m = x.shape[0]
    h = sigmoid(np.matmul(x, theta))
    t1 = np.matmul(-y.T, np.log(h))
    t2_a = (1 - y.T)
    t2_b = np.log(np.clip(1 - h, 0.000000000000001, np.max(1 - h)))  # Used to get numerical issues
    t2 = np.matmul(t2_a, t2_b)

    return ((t1 - t2) / m)[0]


def vanilla(betas, epochs, lr, train_x, train_y):
    import copy

    theta = copy.deepcopy(betas)

    loss_arr = [0 for _ in range(epochs)]
    for i in range(epochs):
        gradient = calculate_gradient(train_x, train_y, theta, fwd, dbg=False)
        theta = theta - lr * np.squeeze(gradient)
        loss = cost(train_x, train_y, theta)
        if DEBUG:
            # print(f"Grad: {(gradient.squeeze() * lr).tolist()}")
            print(f"Theta: {theta.tolist()}")
        print(f"Iteration: {i} Loss: {loss}")
        loss_arr[i] = loss

    return loss_arr, theta


def nesterov(betas, epochs, lr, mu, train_x, train_y):
    import copy

    phi = copy.deepcopy(betas)
    theta = copy.deepcopy(betas)

    nesterov_loss = [0 for _ in range(epochs)]
    # for i in tqdm.trange(epochs):
    for i in range(epochs):
        gradient = calculate_gradient(train_x, train_y, theta, fwd, dbg=False)
        phi_prime = theta - lr * np.squeeze(gradient)
        if i == 0:
            theta = phi_prime
        else:
            theta = phi_prime + mu * (phi_prime - phi)
        phi = phi_prime
        loss = cost(train_x, train_y, theta)
        if DEBUG:
            print(f"Grad: {(gradient.squeeze() * lr).tolist()}")
            print(f"Theta: {theta.tolist()}")
            print(f"Phi: {phi.tolist()}")
        print(f"Iteration: {i} Loss: {loss}")
        nesterov_loss[i] = loss

    return nesterov_loss, theta, phi


# Callers

In [25]:
losses, weights = vanilla(betas, 10, lr, train_x, train_y)


Theta: [-0.015940504807692282, 0.004660081370379265, -0.018115234375, 0.01513671875, -0.01572265625, 1.578282828284031e-05, -0.0033203125000000003, 0.0005859375000000001, 0.0009765625, 0.0]
Iteration: 0 Loss: 0.68240883327532
Theta: [-0.031588549745657245, 0.008797658420500704, -0.035809652796559045, 0.02958523222229394, -0.030782344042775557, -0.00034631701151048615, -0.006198271247921871, 0.0006561815513320045, 0.0017152367103748095, 0.0005105740436275761]
Iteration: 1 Loss: 0.6723754870271551
Theta: [-0.046949521297982175, 0.01247970505186553, -0.05309863527278311, 0.04338933125949955, -0.045229765522874284, -0.001028415157952132, -0.008690928450902899, 0.00028167972350913613, 0.0022594974386068918, 0.0014606141285014767]
Iteration: 2 Loss: 0.6629647982176898
Theta: [-0.06202908232831777, 0.01576520256582198, -0.06999683984169393, 0.056589264959550106, -0.05911065548084562, -0.00197977709643906, -0.010848566314179332, -0.0004751963750072131, 0.0026470025389547843, 0.0027876348247405

In [ ]:
losses, theta, phi = nesterov(betas, 10, lr, mu, train_x, train_y)